**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

Changes in yolov3_training.cfg file in cfg folder:
*   uncomment training batch size with value 64 (line 6) and training subdivisions with value 16 (line 7)
*   comment testing batch size with value 1 (line 3) and testing subdivisions with value 1 (line 4)
*   change classes from 80 to 1 on lines 610, 696, 783
*   change filters from 255 to 18 ((number of classes + 5) * 3) on lines 603, 689, 776



In [ ]:
#Change classes from 80 to 1
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
#Change filters from 255 to 18 
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
#Create "obj.names" file in "data" folder
!echo "ruler" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract images and prepare dataset** 

In [ ]:
#Unzip images from file "dataset.zip" in main folder on drive to "obj" folder
!unzip /mydrive/dataset.zip -d data/obj

In [ ]:
# It converts the class index on the .txt files. We works with only one class, so if the index is different from 0 then it has to be changed.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
#If the image format is different from .jpg, then it has to be changed to .jpg or remake this code.
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)
print(len(images_list))

In [ ]:
#Create train.txt file
ratio = 0.9 #Test vs valid ratio
file = open("data/train.txt", "w") 
max_train = int(ratio*len(images_list))
file.write("\n".join(images_list[0:max_train])) 
file.close() 

In [ ]:
#Create test.txt file
file = open("data/test.txt", "w") 
file.write("\n".join(images_list[max_train:len(images_list)])) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

In [ ]:
# Retrain weights
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3_ruler.weights -dont_show 

**7) Evaluate trained model**

In [ ]:
# Evaluate model with mAP metrics
!./darknet detector map data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3_ruler.weights

In [ ]:
# Evaluate model with recall metrics
!./darknet detector recall data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3_ruler.weights